# NX-OS NETCONF Practice Walkthrough

## Explore YANG Models and Review Sample NX-OS NETCONF Python `ncclient` Code
---



### Explore YANG Models

* The NX-OS YANG models are somewhat extensive and there tools available to help you navigate the model trees:
 1. [`pyang`](https://github.com/mbj4668/pyang)
  - Pros - simple, CLI tool, simple installation (via [Python pip](https://pypi.org/project/pyang/)), easy to use.
  - Cons - Can be challenging to navigate through large models.
 2. [YANG Suite](https://developer.cisco.com/yangsuite/)
  - Pros - friendly GUI tool, variety of installation options ([Docker](https://developer.cisco.com/docs/yangsuite/) and [Python pip](https://github.com/CiscoDevNet/yangsuite)), easy to learn and use.
  - Cons - Newer product which has some UI bugs.
 
 **Note:** This walkthrough contains sample NETCONF payloads generated with YANG Suite although the walkthrough does not describe how to use YANG Suite.  The walkthrough will include NETCONF payloads from the files in the [netconf_payloads](/lab/workspaces/auto-I/tree/resources/nexus/netconf/netconf_practice_walkthrough/netconf_payloads) directory.  You may use the NETCONF payload files for reference although you do not need to review them directly.
 
---



###  Interact with NX-OS YANG Modules Using Python **ncclient**

* **ncclient** is a Python NETCONF client.
* Not part of the Python Standard Library - install with **pip**:

```shell
# Already installed in the lab environment
pip install ncclient
```

---
### Import the 'manager' class from the 'ncclient' library & the 'xmltodict' module

In [1]:
from ncclient import manager
import xmltodict

In [2]:
device = {
    'host': '10.10.20.58',
    'port': 830,
    'username': 'admin',
    'password': 'Cisco123',
    'hostkey_verify': 'False',
    'device_params': {
        'name': 'nexus'
    }
}

---
### Get Nexus NETCONF capabilities

In [3]:
with manager.connect(**device) as m:
    caps = m.server_capabilities

header_msg = f'{len(caps)} availabile NETCONF capabilities'
header_border = (f'{"-" * len(header_msg)}')

---
### Print NETCONF capabilities

In [4]:
print(f'\n{header_border}\n'
        f'{header_msg}\n'
        f'{header_border}\n')

for i, c in enumerate(caps):        
    print(f'{i + 1}. {c}')


----------------------------------
38 availabile NETCONF capabilities
----------------------------------

1. urn:ietf:params:netconf:base:1.0
2. urn:ietf:params:netconf:base:1.1
3. urn:ietf:params:netconf:capability:writable-running:1.0
4. urn:ietf:params:netconf:capability:rollback-on-error:1.0
5. urn:ietf:params:netconf:capability:candidate:1.0
6. urn:ietf:params:netconf:capability:validate:1.1
7. urn:ietf:params:netconf:capability:confirmed-commit:1.1
8. urn:ietf:params:netconf:capability:notification:1.0
9. urn:ietf:params:netconf:capability:interleave:1.0
10. urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=report-all
11. http://cisco.com/ns/yang/cisco-nx-os-device?revision=2020-07-20&module=Cisco-NX-OS-device
12. http://openconfig.net/yang/acl?revision=2019-11-27&module=openconfig-acl&deviations=cisco-nx-openconfig-acl-deviations
13. http://openconfig.net/yang/bfd?revision=2019-10-25&module=openconfig-bfd&deviations=cisco-nx-openconfig-bfd-deviations
14. http://op

---
### Set the XML payload base path

In [5]:
payload_base_path = 'netconf_payloads/'
native = '1_native/'
openconfig = '2_openconfig/'

---
### Configure dictionary of Native & OpenConfig YANG XML payloads

In [6]:
payloads = {
    'Native - Get Serial Number': '1_serial_num_payload.xml',
    'Native - Get Ethernet1/1 Oper': '2_get_eth1_1_payload.xml',
    'Native - Get Ethernet1/1 Config': '3_get_config_eth1_1_payload.xml',
    'Native - Configure Ethernet1/10': '4_edit_config_eth1_2_payload.xml',
    'Native - Get Ethernet 1/10 Config': '5_get_config_eth1_2_payload.xml',
    'Native - Configure Web Server ACL': '6_native_web_server_acl_payload.xml',
    'Native - Copy Run Start': '7_copy_run_start_payload.xml',
    'OpenConfig - Configure Web Server ACL Seq. 10': '1_oc_create_seq_10_payload.xml',
    'OpenConfig - Configure Web Server ACL Seq. 20': '1a_oc_create_seq_20_payload.xml',
    'OpenConfig - Configure Web Server Int. Access Group': '2_oc_apply_e1_2_access_group_payload.xml',
    'OpenConfig - Get Web ACL Config': '3_oc_acl_get_config_payload.xml'
}

for k, v in payloads.items():
    print(f'{k:<50} >> {v}')

Native - Get Serial Number                         >> 1_serial_num_payload.xml
Native - Get Ethernet1/1 Oper                      >> 2_get_eth1_1_payload.xml
Native - Get Ethernet1/1 Config                    >> 3_get_config_eth1_1_payload.xml
Native - Configure Ethernet1/10                    >> 4_edit_config_eth1_2_payload.xml
Native - Get Ethernet 1/10 Config                  >> 5_get_config_eth1_2_payload.xml
Native - Configure Web Server ACL                  >> 6_native_web_server_acl_payload.xml
Native - Copy Run Start                            >> 7_copy_run_start_payload.xml
OpenConfig - Configure Web Server ACL Seq. 10      >> 1_oc_create_seq_10_payload.xml
OpenConfig - Configure Web Server ACL Seq. 20      >> 1a_oc_create_seq_20_payload.xml
OpenConfig - Configure Web Server Int. Access Group >> 2_oc_apply_e1_2_access_group_payload.xml
OpenConfig - Get Web ACL Config                    >> 3_oc_acl_get_config_payload.xml


---
### Setup Nexus **get** serial number NETCONF filter

In [7]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}1_serial_num_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get' key from the dictionary & convert to XML 
nc_data = data['rpc']['get']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

In [8]:
from pprint import pprint as pp
print(raw_filter)
print()
pp(data)
print()
pp(nc_data)
print()
print(filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <serial/>
      </System>
    </filter>
  </get>
</rpc>

{'rpc': {'@message-id': '101',
         '@xmlns': 'urn:ietf:params:xml:ns:netconf:base:1.0',
         'get': {'filter': {'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device',
                                       'serial': None}}}}}

{'filter': {'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device',
                       'serial': None}}}

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<serial></serial>
	</System>
</filter>


---
### Print YANG Explorer XML output

In [9]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <serial/>
      </System>
    </filter>
  </get>
</rpc>


---
### Print ncclient filter

In [10]:
print(filter)

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<serial></serial>
	</System>
</filter>


---
### Send NETCONF **get** request

In [11]:
with manager.connect(**device) as m:
    response = m.get(filter=filter)

---
### Print NETCONF reply

In [12]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:55f36536-f231-4496-b35e-2b7b7cc6439f">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
            <serial>91OMZP37WBR</serial>
        </System>
    </data>
</rpc-reply>



---
---
### Setup Ethernet1/1 **get** operational data NETCONF filter

In [13]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}2_get_eth1_1_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get' key from the dictionary & convert to XML 
nc_data = data['rpc']['get']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [14]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <intf-items>
          <phys-items>
            <PhysIf-list>
              <id>eth1/1</id>
            </PhysIf-list>
          </phys-items>
        </intf-items>
      </System>
    </filter>
  </get>
</rpc>


---
### Print ncclient filter

In [15]:
manager.connect?

Signature: manager.connect(*args, **kwds)
Docstring: <no docstring>
File:      /usr/local/lib/python3.9/site-packages/ncclient/manager.py
Type:      function


In [16]:
device = {
    'host': '10.10.20.58',
    'port': 830,
    'username': 'admin',
    'password': 'Cisco123',
    'hostkey_verify': False,
    'device_params': {
        'name': 'nexus'
    }
}

In [17]:
dir(device)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [18]:
with manager.connect(**device) as conn:
    r = conn.get(filter=filter)

In [19]:
pp(xmltodict.parse(r.xml, dict_constructor=dict))

{'rpc-reply': {'@message-id': 'urn:uuid:6af7816f-1d1d-4eaf-b199-6e9a434c3d09',
               '@xmlns': 'urn:ietf:params:xml:ns:netconf:base:1.0',
               '@xmlns:if': 'http://www.cisco.com/nxos:1.0:if_manager',
               '@xmlns:nfcli': 'http://www.cisco.com/nxos:1.0:nfcli',
               '@xmlns:nxos': 'http://www.cisco.com/nxos:1.0',
               '@xmlns:vlan_mgr_cli': 'http://www.cisco.com/nxos:1.0:vlan_mgr_cli',
               'data': {'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device',
                                   'intf-items': {'phys-items': {'PhysIf-list': {'FECMode': 'auto',
                                                                                 'accessVlan': 'vlan-1',
                                                                                 'adminSt': 'down',
                                                                                 'aggrmbrif-items': {'bdlPortNum': '0',
                                               

In [20]:
print(filter)

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<intf-items>
			<phys-items>
				<PhysIf-list>
					<id>eth1/1</id>
				</PhysIf-list>
			</phys-items>
		</intf-items>
	</System>
</filter>


---
### Send NETCONF **get** request

In [21]:
with manager.connect(**device) as m:
    response = m.get(filter=filter)

---
### Print NETCONF reply

In [22]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:8242485d-89fb-4a73-b2f5-f68f6f74bdd7">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
            <intf-items>
                <phys-items>
                    <PhysIf-list>
                        <id>eth1/1</id>
                        <FECMode>auto</FECMode>
                        <accessVlan>vlan-1</accessVlan>
                        <adminSt>down</adminSt>
                        <aggrmbrif-items>
                            <bdlPortNum>0</bdlPortNum>
                            <channelingSt>unknown</channelingSt>
                            <flags/>
                            <ltlProgrammed>false</ltlProgrammed>
         

---
---
### Setup Ethernet1/1 **get-config** NETCONF filter

In [23]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}3_get_config_eth1_1_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['get-config']

# Remove the 'source' item
del nc_data['source']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [24]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get-config>
    <source>
      <running/>
    </source>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <ipv4-items>
          <inst-items>
            <dom-items>
              <Dom-list>
                <if-items>
                  <If-list>
                    <id>eth1/1</id>
                  </If-list>
                </if-items>
              </Dom-list>
            </dom-items>
          </inst-items>
        </ipv4-items>
      </System>
    </filter>
  </get-config>
</rpc>


---
### Print ncclient filter

In [25]:
print(filter)

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<ipv4-items>
			<inst-items>
				<dom-items>
					<Dom-list>
						<if-items>
							<If-list>
								<id>eth1/1</id>
							</If-list>
						</if-items>
					</Dom-list>
				</dom-items>
			</inst-items>
		</ipv4-items>
	</System>
</filter>


In [26]:
device = {
    'host': '10.10.20.58',
    'port': 830,
    'username': 'admin',
    'password': 'Cisco123',
    'hostkey_verify': False,
    'device_params': {
        'name': 'nexus'
    }
}

In [27]:
with manager.connect(**device) as conn:
    r = conn.get_config(source='running', filter=filter)

In [28]:
print(dir(r))
pp(xmltodict.parse(r.data_xml, dict_constructor=dict))

['ERROR_CLS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_data', '_errors', '_huge_tree', '_parsed', '_parsing_error_transform', '_parsing_hook', '_raw', '_root', 'data', 'data_ele', 'data_xml', 'error', 'errors', 'ok', 'parse', 'set_parsing_error_transform', 'xml']
{'data': {'@xmlns': 'urn:ietf:params:xml:ns:netconf:base:1.0',
          '@xmlns:if': 'http://www.cisco.com/nxos:1.0:if_manager',
          '@xmlns:nfcli': 'http://www.cisco.com/nxos:1.0:nfcli',
          '@xmlns:nxos': 'http://www.cisco.com/nxos:1.0',
          '@xmlns:vlan_mgr_cli': 'http://www.cisco.com/nxos:1.0:vlan_mgr_cli',
          'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device'}}}


---
### Send NETCONF **get-config** request

In [29]:
with manager.connect(**device) as m:
    response = m.get_config(source="running", filter=filter)

---
### Print NETCONF reply

In [30]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:3cdc0832-3efd-4c83-98e3-1b406f74b23a">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device"/>
    </data>
</rpc-reply>



---
---
### Setup Ethernet1/10 **edit-config** NETCONF filter

In [31]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}4_edit_config_eth1_2_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [32]:
print(raw_config)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <edit-config>
    <target>
      <running/>
    </target>
    <config>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <intf-items>
          <phys-items>
            <PhysIf-list>
              <id>eth1/10</id>
              <adminSt>up</adminSt>
              <layer>Layer3</layer>
              <userCfgdFlags>admin_layer,admin_state</userCfgdFlags>
            </PhysIf-list>
          </phys-items>
        </intf-items>
        <ipv4-items>
          <inst-items>
            <dom-items>
              <Dom-list>
                <name>default</name>
                <if-items>
                  <If-list>
                    <id>eth1/10</id>
                    <addr-items>
                      <Addr-list>
                        <addr>192.168.5.2/24</addr>
                        <pref>0</pref>
                        <tag>0</tag>
                      </Addr-list>
                    </

---
### Print ncclient filter

In [33]:
print(config)

<config>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<intf-items>
			<phys-items>
				<PhysIf-list>
					<id>eth1/10</id>
					<adminSt>up</adminSt>
					<layer>Layer3</layer>
					<userCfgdFlags>admin_layer,admin_state</userCfgdFlags>
				</PhysIf-list>
			</phys-items>
		</intf-items>
		<ipv4-items>
			<inst-items>
				<dom-items>
					<Dom-list>
						<name>default</name>
						<if-items>
							<If-list>
								<id>eth1/10</id>
								<addr-items>
									<Addr-list>
										<addr>192.168.5.2/24</addr>
										<pref>0</pref>
										<tag>0</tag>
									</Addr-list>
								</addr-items>
							</If-list>
						</if-items>
					</Dom-list>
				</dom-items>
			</inst-items>
		</ipv4-items>
	</System>
</config>


In [34]:
config = """
<config>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<intf-items>
			<phys-items>
				<PhysIf-list>
					<id>eth1/10</id>
					<adminSt>up</adminSt>
					<layer>Layer3</layer>
					<userCfgdFlags>admin_layer,admin_state</userCfgdFlags>
				</PhysIf-list>
			</phys-items>
		</intf-items>
		<ipv4-items>
			<inst-items>
				<dom-items>
					<Dom-list>
						<name>default</name>
						<if-items>
							<If-list>
								<id>eth1/25</id>
								<addr-items>
									<Addr-list>
										<addr>192.168.5.2/24</addr>
										<pref>0</pref>
										<tag>0</tag>
									</Addr-list>
								</addr-items>
							</If-list>
						</if-items>
					</Dom-list>
				</dom-items>
			</inst-items>
		</ipv4-items>
	</System>
</config>
""".strip()

---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration interface state
show run interface Ethernet1/2
```

In [35]:
with manager.connect(**device) as m:
    r = m.edit_config(target='candidate', config=config)

In [36]:
print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:c77d8945-76f9-4c38-b087-5c25c6ac4da2">
    <ok/>
</rpc-reply>



---
### Send NETCONF **edit-config** request

In [37]:
with manager.connect(**device) as m:
    response = m.edit_config(target="candidate", config=config)

---
### Print NETCONF reply

In [38]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:5310227a-ea8c-4b46-92de-80a34aee3907">
    <ok/>
</rpc-reply>



---
### Commit **candidate** datastore to **running** datastore

In [39]:
with manager.connect(**device) as m:
    response = m.commit()

---
### Print NETCONF reply

In [40]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:89f991c7-6ea0-4044-bbbf-9d7fa4a88758">
    <ok/>
</rpc-reply>



---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration interface state
show run interface Ethernet1/10
```

---
---
### Setup Ethernet1/10 **get-config** NETCONF filter

In [41]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}5_get_config_eth1_2_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['get-config']

# Remove the 'source' item
del nc_data['source']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [42]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get-config>
    <source>
      <running/>
    </source>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <intf-items>
          <phys-items>
            <PhysIf-list>
              <id>eth1/10</id>
            </PhysIf-list>
          </phys-items>
        </intf-items>
        <ipv4-items>
          <inst-items>
            <dom-items>
              <Dom-list>
                <name>default</name>
                <if-items>
                  <If-list>
                    <id>eth1/10</id>
                  </If-list>
                </if-items>
              </Dom-list>
            </dom-items>
          </inst-items>
        </ipv4-items>
      </System>
    </filter>
  </get-config>
</rpc>



---
### Print ncclient filter

In [43]:
print(filter)

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<intf-items>
			<phys-items>
				<PhysIf-list>
					<id>eth1/10</id>
				</PhysIf-list>
			</phys-items>
		</intf-items>
		<ipv4-items>
			<inst-items>
				<dom-items>
					<Dom-list>
						<name>default</name>
						<if-items>
							<If-list>
								<id>eth1/10</id>
							</If-list>
						</if-items>
					</Dom-list>
				</dom-items>
			</inst-items>
		</ipv4-items>
	</System>
</filter>


---
### Send NETCONF **get-config** request

In [44]:
with manager.connect(**device) as m:
    response = m.get_config(source="running", filter=filter)

---
### Print NETCONF reply

In [45]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:5dd72e5c-7fa0-40e2-a1ef-1365ad50f101">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
            <intf-items>
                <phys-items>
                    <PhysIf-list>
                        <id>eth1/10</id>
                        <FECMode>auto</FECMode>
                        <accessVlan>unknown</accessVlan>
                        <adminSt>up</adminSt>
                        <autoNeg>on</autoNeg>
                        <beacon>off</beacon>
                        <bw>0</bw>
                        <controllerId/>
                        <delay>1</delay>
                        <dfeAdaptiveTuning>enable</dfeAdaptiveTuni

---
---

### Install OpenConfig YANG Modules RPM on Nexus 9K Switch

#### Before you continue, you must **manually** install the OpenConfig YANG modules on the Nexus 9K switch

1. SSH to the Nexus switch and determine the software version with the `show version` command.
2. Navigate to the [Cisco DevNet Artifactory](https://devhub.cisco.com/artifactory/open-nxos-agents/) site and choose the folder which matches the software version of your Nexus 9K.
3. Navigate the folder structure and download the RPM file for OpenConfig YANG modules.
    * The file for NX-OS version 9.3(5) is [mtx-openconfig-all-1.0.0.0-9.3.5.lib32_n9000.rpm](https://devhub.cisco.com/artifactory/open-nxos-agents/9.3-5/x86_64/mtx-openconfig-all-1.0.0.0-9.3.5.lib32_n9000.rpm).
4. Use SCP to copy the OpenConfig YANG Module RPM file to the Nexus 9K.

```bash
# Example command to SCP the RPM file from your 'Downloads' directory to the Nexus 9K
scp ~/Downloads/mtx-openconfig-all-1.0.0.0-9.3.5.lib32_n9000.rpm admin@10.10.20.58:mtx-openconfig-all-1.0.0.0-9.3.5.lib32_n9000.rpm
```

5. SSH to the Nexus 9K and use the following commands to enter the **Bash** shell:

```bash
! Enter configuration mode
configure terminal

! Enable the Bash shell
feature bash-shell

! Enter the Bash shell as the 'root' user
run bash sudo su -

! Install the OpenConfig YANG Model RPM on NX-OS version 9.x and earlier
yum install -y /bootflash/mtx-openconfig-all-1.0.0.0-9.3.5.lib32_n9000.rpm

! Install the OpenConfig YANG Model RPM on NX-OS version 10.x and later
dnf install -y /bootflash/mtx-openconfig-all-1.0.0.0-9.3.5.lib32_n9000.rpm

```

In [46]:
# Pause Jupyter run and wait for manual confirmation of OpenConfig YANG model RPM installation
input('Press Return/Enter to continue: ')

Press Return/Enter to continue:  


''

### Setup OpenConfig web server ACL, sequence 10 **edit-config** NETCONF filter

In [47]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}1_oc_create_seq_10_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'edit-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [48]:
print(raw_config)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <edit-config>
    <target>
      <running/>
    </target>
    <config>
      <acl xmlns="http://openconfig.net/yang/acl">
        <acl-sets>
          <acl-set>
            <name>WEB-SERVER</name>
            <type>ACL_IPV4</type>
            <acl-entries>
              <acl-entry>
                <sequence-id>10</sequence-id>
                <config>
                  <sequence-id>10</sequence-id>
                </config>
                <ipv4>
                  <config>
                    <source-address>192.168.2.0/24</source-address>
                    <destination-address>192.168.5.3/32</destination-address>
                    <protocol>IP_TCP</protocol>
                  </config>
                </ipv4>
                <transport>
                  <config>
                    <destination-port>80</destination-port>
                  </config>
                </transport>
                <actions>
     

---
### Print ncclient filter

In [49]:
print(config)

<config>
	<acl xmlns="http://openconfig.net/yang/acl">
		<acl-sets>
			<acl-set>
				<name>WEB-SERVER</name>
				<type>ACL_IPV4</type>
				<acl-entries>
					<acl-entry>
						<sequence-id>10</sequence-id>
						<config>
							<sequence-id>10</sequence-id>
						</config>
						<ipv4>
							<config>
								<source-address>192.168.2.0/24</source-address>
								<destination-address>192.168.5.3/32</destination-address>
								<protocol>IP_TCP</protocol>
							</config>
						</ipv4>
						<transport>
							<config>
								<destination-port>80</destination-port>
							</config>
						</transport>
						<actions>
							<config>
								<forwarding-action>ACCEPT</forwarding-action>
							</config>
						</actions>
					</acl-entry>
				</acl-entries>
			</acl-set>
		</acl-sets>
	</acl>
</config>


---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration ACL state
show run | section access-list
```

In [50]:
m = manager.connect(**device)

In [51]:
m.lock(target='candidate')

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d0487e8e-4a7c-4aee-b5cf-2309a14294b4">
    <ok/>
</rpc-reply>

In [52]:
m.edit_config(target='candidate', config=config)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:a5b12f11-e354-4b88-801f-c5b833ff854e">
    <ok/>
</rpc-reply>

In [53]:
m.unlock()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:1a924c24-5d27-43f2-8469-8f781c4d5d6f">
    <ok/>
</rpc-reply>

In [54]:
m.commit()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:2f408051-1a6a-4100-8aa1-6f405e308cce">
    <ok/>
</rpc-reply>

---
### Lock the **candidate** datastore and send the **edit-config** NETCONF request

In [55]:
with manager.connect(**device) as m:
    m.lock()
    response = m.edit_config(target="candidate", config=config)
    m.unlock()

---
### Print NETCONF reply

In [56]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d25b296a-e534-41fb-b84e-cc411a2c7f5d">
    <ok/>
</rpc-reply>



---
### Commit the **candidate** datastore to the **running** datastore

In [57]:
with manager.connect(**device) as m:
    response = m.commit()

---
### Print NETCONF reply

In [58]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:8044b4c9-5300-462f-8979-e403204d0012">
    <ok/>
</rpc-reply>



---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration ACL state
show run | section access-list
```

---
---
### Setup OpenConfig web server ACL, sequence 20 **edit-config** NETCONF filter

In [59]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}1a_oc_create_seq_20_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'edit-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [60]:
print(raw_config)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <edit-config>
    <target>
      <running/>
    </target>
    <config>
      <acl xmlns="http://openconfig.net/yang/acl">
        <acl-sets>
          <acl-set>
            <name>WEB-SERVER</name>
            <type>ACL_IPV4</type>
            <acl-entries>
              <acl-entry>
                <sequence-id>20</sequence-id>
                <config>
                  <sequence-id>20</sequence-id>
                </config>
                <ipv4>
                  <config>
                    <source-address>192.168.2.0/24</source-address>
                    <destination-address>192.168.5.3/32</destination-address>
                    <protocol>IP_ICMP</protocol>
                  </config>
                </ipv4>
                <actions>
                  <config>
                    <forwarding-action>ACCEPT</forwarding-action>
                  </config>
                </actions>
              </acl-entry>
 

---
### Print ncclient filter

In [61]:
print(config)

<config>
	<acl xmlns="http://openconfig.net/yang/acl">
		<acl-sets>
			<acl-set>
				<name>WEB-SERVER</name>
				<type>ACL_IPV4</type>
				<acl-entries>
					<acl-entry>
						<sequence-id>20</sequence-id>
						<config>
							<sequence-id>20</sequence-id>
						</config>
						<ipv4>
							<config>
								<source-address>192.168.2.0/24</source-address>
								<destination-address>192.168.5.3/32</destination-address>
								<protocol>IP_ICMP</protocol>
							</config>
						</ipv4>
						<actions>
							<config>
								<forwarding-action>ACCEPT</forwarding-action>
							</config>
						</actions>
					</acl-entry>
				</acl-entries>
			</acl-set>
		</acl-sets>
	</acl>
</config>


---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration ACL state
show run | section access-list
```

---
### Lock the **running** datastore and send the **edit-config** NETCONF request directly to the running configuration

In [62]:
with manager.connect(**device) as m:
    m.lock()
    response = m.edit_config(target="running", config=config)
    m.unlock()

---
### Print NETCONF reply

In [63]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d2bc9a95-ba06-41b9-ae6b-164649f77e41">
    <ok/>
</rpc-reply>



---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration ACL state
show run | section access-list
```

---
---
### Setup OpenConfig web server ACL, Ethernet1/10 access group outbound **edit-config** NETCONF filter

In [65]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}2_oc_apply_e1_10_access_group_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'edit-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [66]:
print(raw_config)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <edit-config>
    <target>
      <running/>
    </target>
    <config>
      <acl xmlns="http://openconfig.net/yang/acl">
        <interfaces>
          <interface>
            <id>eth1/10</id>
            <config>
              <id>eth1/10</id>
            </config>
            <interface-ref>
              <config>
                <interface>eth1/10</interface>
              </config>
            </interface-ref>
            <egress-acl-sets>
              <egress-acl-set>
                <set-name>WEB-SERVER</set-name>
                <type>ACL_IPV4</type>
                <config>
                  <set-name>WEB-SERVER</set-name>
                  <type>ACL_IPV4</type>
                </config>
              </egress-acl-set>
            </egress-acl-sets>
          </interface>
        </interfaces>
      </acl>
    </config>
  </edit-config>
</rpc>


---
### Print ncclient filter

In [67]:
print(config)

<config>
	<acl xmlns="http://openconfig.net/yang/acl">
		<interfaces>
			<interface>
				<id>eth1/10</id>
				<config>
					<id>eth1/10</id>
				</config>
				<interface-ref>
					<config>
						<interface>eth1/10</interface>
					</config>
				</interface-ref>
				<egress-acl-sets>
					<egress-acl-set>
						<set-name>WEB-SERVER</set-name>
						<type>ACL_IPV4</type>
						<config>
							<set-name>WEB-SERVER</set-name>
							<type>ACL_IPV4</type>
						</config>
					</egress-acl-set>
				</egress-acl-sets>
			</interface>
		</interfaces>
	</acl>
</config>


---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration interface state
show run interface Ethernet1/10
```

---
### Send the NETCONF request directly to the **running** datastore, without lock

In [68]:
with manager.connect(**device) as m:
    response = m.edit_config(target="running", config=config)

---
### Print NETCONF reply

In [69]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:5945062a-e478-45cc-b6c9-4779ca8a5e2b">
    <ok/>
</rpc-reply>



---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration interface state
show run interface Ethernet1/10
```

---
### Secondary post-check Nexus configuration via **Terminal**

```shell
# Verify ICMP & TCP connectivity to the web server (192.168.5.3)
ping -c 4 192.168.5.3

curl -i http://192.168.5.3
```

---
---
### Setup OpenConfig web server ACL **get** NETCONF filter

In [70]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}3_oc_acl_get_config_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['get-config']

# Remove the 'source' item
del nc_data['source']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [71]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get-config>
    <source>
      <running/>
    </source>
    <filter>
      <acl xmlns="http://openconfig.net/yang/acl"/>
    </filter>
  </get-config>
</rpc>


---
### Print ncclient filter

In [72]:
print(filter)

<filter>
	<acl xmlns="http://openconfig.net/yang/acl"></acl>
</filter>


---
### Send NETCONF **get** request

In [73]:
with manager.connect(**device) as m:
    response = m.get(filter=filter)

---
### Print NETCONF reply

In [74]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:abe5a744-a857-4dbd-ba4e-c96260f5fc5b">
    <data>
        <acl xmlns="http://openconfig.net/yang/acl">
            <interfaces>
                <interface>
                    <id>eth1/10</id>
                    <config>
                        <id>eth1/10</id>
                    </config>
                    <egress-acl-sets>
                        <egress-acl-set>
                            <set-name>WEB-SERVER</set-name>
                            <type>ACL_IPV4</type>
                            <config>
                                <set-name>WEB-SERVER</set-name>
                                <type>ACL_IPV4</type>
                            </confi

---
---
### Setup copy of the running **datastore** to the startup **datastore** NETCONF payload

In [75]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}7_copy_run_start_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_payload = payload.read()
    data = xmltodict.parse(raw_payload,
                           dict_constructor=dict)

# Parse the 'rpc' key from the dictionary & convert to XML 
nc_data = data['rpc']

# Remove the '@message-id' and @xmlns items
del nc_data['@message-id']
del nc_data['@xmlns']
payload = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [76]:
print(raw_payload)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <copy_running_config_src xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <startup-config/>
  </copy_running_config_src>
</rpc>


---
### Print ncclient payload

In [77]:
print(payload)

<copy_running_config_src xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
	<startup-config></startup-config>
</copy_running_config_src>


---
### Pre-check Nexus **running** vs **startu**p configuration via **vshell**

```shell
# Verify the pre-copy interface & ACL state
show run interface Ethernet1/10

show run | sec access-list
```

---
### Send NETCONF **dispatch** request

In [78]:
# Parse the root node from the XML payload
from lxml.etree import fromstring
save_rpc = fromstring(payload)

# Alternate payload format
"""
save_rpc = '''
<copy xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <source>running</source>
    <destination>startup</destination>
</copy>
'''
"""

# Send NETCONF request
with manager.connect(**device) as m:
    response = m.dispatch(save_rpc)

---
### Print NETCONF reply

In [79]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:328b7160-ee45-4f8c-ae7c-b39414eb855d">
    <ok/>
</rpc-reply>



---
### Post-check Nexus **running** vs **startu**p configuration via **vshell**

```shell
# Verify the post-copy interface & ACL state
show run interface Ethernet1/10

show run | sec access-list
```

---